## README file for ``DecayGraphs.py`` and ``RunDecayGraphs.ipynb``

* ``RunDecayGraphs.ipynb`` installs ``DecayGraphs.py`` and calls three functions from ``DecayGraphs.py`` to make three graphs: a plot of branching ratio over mass, a contour plot of lifetime with respect to mixing angle and mass of the sterile neutrino, and a spectrum of neutrino decay products from the sterile decay.
* Download and install DES as a package if you haven't already:
1. Navigate to the directory that your SterileDecayGraphs folder is in. Make a folder there called DES, then make a folder within DES that is also named DES.
2. Download ``Differential_Equation_Solver_new.py`` from my GitHub page at https://github.com/hannahrasmussen/DES and put it in the folder within the folder.
3. In the folder (not the folder within the folder), put a setup.py folder with the following code:
        from setuptools import setup
        setup(name='DES',
          version='0.1',
          description='Code to solve ODEs',
          url='https://github.com/hannahrasmussen/DES',
          author='Hannah Rasmussen',
          author_email='hannahrasmussen17@gmail.com',
          license='MIT',
          packages=['DES'],
          zip_safe=False)
4. Now, go to the terminal and checkout your new folder (not the folder within a folder) using the ‘cd’ command. 
5. Once you’re there, type and enter ‘pip install -e .’ (including the period). That ‘-e’ extension means that updates you make to the code are automatically also installed, which is obviously what we want.
6. Now, to see if it worked, restart your jupyter notebook kernel in ``RunDecayGraphs.ipynb`` and try and do ‘from DES import Differential_Equation_Solver_new.py’. If there are no errors, you can delete that line and should be good to run the code as is.

# ``RunDecayGraphs.ipynb``

* The first window installs ``DecayGraphs.py`` as ``DG``. See below for details on ``DecayGraphs.py``.
* The seconds window calls the function ``graph_BR`` and passes the mixing angle $5 \times 10^{-5}$ to this function. The graph does not change for different mixing angles, a mixing angle is simply needed to run the decay rate functions from the ``graph_BR`` function. That is, mixing angle changes the rate of each decay (proportional to $\sin^2(\theta)$), but does not change the relative abundance of each decay.
* The third window call the function ``graph_mix_ms_t``, with 250 MeV as the lower bound for sterile neutrino mass, 350 MeV as the upper bound for sterile neutrino mass, $1 \times 10^{-5}$ as the lower bound for mixing angle, and $1 \times 10^{-4}$ as the upper bound for mixing angle. Our model should be accurate for a much wider range of mixing angles, and for masses of 150-500 MeV, but we chose this range since we have only tested masses of 300 MeV thus far and models with mixing angles beyond this range for $m_s=300$ MeV are not noteworthy (either decay too early to have an effect or too late that N-effective blows up).
* The fourth and final window calls the function ``graph_final_num`` and passes the arguments of $m_s=300$ MeV and $\theta=5 \times 10^{-5}$. Changing mixing angle will change the height of the graph, and changing mass will obviously change the graph's width, but it will also affect the shape of the graph by changing the relative heights of the curves of the various decay paths.

# ``DecayGraphs.py``

## Imports

* ``numpy`` is a python package that allows for quick and easy computing using multi-dimensional arrays.
* ``matplotlib.pyplot`` is a python package that provides a wide variety of 2-dimensional plotting options. 
* ``cm`` is a module from ``matplotlib`` that allows for more functionality with coloring plots. I use it to designate colors for my contour plot.
* ``DES`` is an ODE solver package that contains ``Differential_Equation_Solver_new.py``, which solves differential equations using a Fourth-Order Runge Kutta routine with an adaptive stepsize. See the README file in my DES repo on GitHub for more documentation on this code.

## Physical Constants

* ``alpha``: The unitless fine structure constant, $\alpha = 1/137$ 
* ``dtda_part2``:  Used in driver functions but can be calculated out here, $\frac{2\pi}{3} $ 
* ``f_pi``: Pion decay constant, $f_{\pi} = 131 \text{ MeV}$
* ``Gf``: Fermi constant, $G_F = 1.166 \times 10 ^{-11} \text{ MeV}^{-2}$
* ``me``: Electron mass, $m_e = 0.511 \text{ MeV}$
* ``mpi_neutral``: Neutral pion mass, $m_{\pi^0} = 135 \text{ MeV}$
* ``mpi_charged``: Charged pion mass, $m_{\pi^{\pm}} = 139.569 \text{ MeV}$
* ``mPL``: Planck mass, $m_{\rm Pl} = 1.124 \times 10^{22} \text{ MeV}$
* ``mu``: Muon mass, $m_{\mu} = 105.661 \text{ MeV}$
* ``eps_e``: Constant used in several integrations, $\epsilon_e = \frac{m_e}{m_{\mu}}$
* ``Enumax``: Maximum energy of the muon and electron neutrino from the decay of the muon in the rest frame of the muon, $E_{\nu \rm max} = \frac{m_{\mu}}{2}(1-\epsilon_e^2)$

## Model-specific parameters

* ``n``: Number of steps for Gauss-Laguerre and Gauss-Legendre quadrature, max is about 150 before it stops working well
* ``num``: Total number of ODEs that will be solved, that is, total number of $N_E$ ODEs (one for each energy bin) plus two for temperature and time
* ``steps``: Number of steps over independent variable (``a``) that will be saved and returned by DES
* ``d_steps``: Number of steps between main steps over independent variable that will not be returned by DES in an effort to save computational expense but not lose accuracy
* ``a_init``: Initial scale factor 
* ``a_final``: Final scale factor, DES will stop when this value is reached OR when the number of steps is reached, whichever happens first 
* ``y_init``: Initial array of y values, that is, dependent variables. Initial values of $N_E$ are all left as 0 here, initial values for temperature and time are set on the next line.
* ``y_init[-2]``: Initial temperature in MeV, for the duration of the code ``y[-2]`` is temperature
* ``y_init[-1]``: Initial time in units of MeV$^{-1}$, for the duration of the code ``y[-1]`` is time. ($1$ s $= \hbar \times 1.52 \times 10^{21}$ MeV$^{-1}$)
* ``boxsize``: Space accounted for by each energy bin
* ``e_array``: Energy array, each bin accounts for all neutrinos with energies between $E$ and $E +$ ``boxsize``
* Gauss-Laguerre quadrature: A mathematical tool to evaluate any integral from $[0,\infty)$ using Laguerre polynomials. You send $n$ to the function and it returns the $n$ roots of the $n^{\rm th}$ Laguerre polynomial. These $n$ roots are ``x_values``. Each root has a corresponding weight in ``w_values``. By evaluating $\sum_{i=1}^n w_i * e^{-x_i}f(x_i)$, an approximation for $\int_0^{\infty} f(x) dx$ is returned.
* ``x_values``: The $n$ roots of the $n^{\rm th}$ Laguerre polynomial
* ``w_values``: The weights that correspond to ``x_values``
* Gauss-Legendre quadrature: A mathematical tool to evaluate any integral from $[-1,1]$ using Legendre polynomials. You send $n$ to the function and it returns the $n$ roots of the $n^{\rm th}$ Legendre polynomial. These n roots are ``x_valuese``. Each root has a corresponding weight in ``w_valuese``. By evaluating $\sum_{i=1}^n w_i f(x_i)$, an approximation for $\int_{-1}^1 f(x) dx$ is returned. Note that this can be molded to calculate any definite integral, $\int_a^b f(x) dx$.
* ``x_valuese``: The $n$ roots of the $n^{\rm th}$ Legendre polynomial
* ``w_valuese``: The weights that correspond to ``x_valuese``
* ``D``: Parameter that describes the dilution of the sterile neutrinos from the time they decoupled, we change this later
* ``fT``: Fraction of mass converted to thermal energy in the plasma, we really change this later

## Integral-related functions

### ``I1``  - the integrand in $\rho_{e^{\pm}}$
The energy density of electrons and positrons, $\rho_{e^{\pm}}$, is $$ \rho_{e^{\pm}} = \frac{2T^4}{\pi^2}\int_0^\infty \text{I1 }d\xi = \frac{2T^4}{\pi^2}\int_0^\infty\frac{\xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1}d\xi. $$ This function, however, only computes the *integrand* portion of that equation. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficient, $\left(2T^4 \right)/ \pi^2$, is ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $\rho_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $\rho_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``E_den``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} \xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1} $$
    
### ``I2`` - the integrand in $P_{e^{\pm}}$
The pressure of electrons and positrons, $P_{e^{\pm}}$, is $$ P_{e^{\pm}} = \frac{2T^4}{3\pi^2}\int_0^\infty \text{I2 } d\xi = \frac{2T^4}{3\pi^2}\int_0^\infty\frac{e^{\xi} \xi^4}{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} d\xi. $$ This function, however, only computes the *integrand* portion of that equation. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficient, $\left(2T^4 \right)/ \left(3\pi^2 \right)$, is ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $P_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $P_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``Pressure``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} \xi^4 }{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} $$
    
### ``dI1`` - the integrand in $d\rho_{e^{\pm}}/dT$
The derivative of the energy density of electrons and positrons, $d\rho_{e^{\pm}}/dT$, is $$ \frac{d\rho_{e^{\pm}}}{dT} = \frac{8T^3}{\pi^2}\int_0^\infty \text{I1 }d\xi + \frac{2Tm_e^2}{\pi^2} \int_0^{\infty} \text{dI1 } d\xi = \frac{8T^3}{\pi^2}\int_0^\infty\frac{\xi^2\sqrt{\xi^2+x^2}}{e^{\sqrt{\xi^2+x^2}}+1}d\xi + \frac{2Tm_e^2}{\pi^2} \int_0^{\infty} \frac{\sqrt{\xi^2 + x^2}}{e^{\sqrt{\xi^2 + x^2}}+1} d\xi . $$ 

Notice that we already have code for the integrand in the first term, ``I1``!! This function computes the *integrand* portion of the second term, ``dI1``. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficients of both terms, $\left( 8T^3 \right) / \pi^2$ and $ \left( 2Tm_e^2 \right)/ \pi^2$ are both ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $d\rho_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $d\rho_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``dE_den``: The integrand at the given $\xi$ and $x$ value: $$\frac{e^{\xi} \sqrt{\xi^2 + x^2}}{e^{\sqrt{\xi^2 + x^2}}+1}$$
    
### ``dI2`` - the integrand in $dP_{e^{\pm}}/dT$
The derivative of the energy density of electrons and positrons, $d\rho_{e^{\pm}}/dT$, is $$ \frac{d\rho_{e^{\pm}}}{dT} = \frac{8T^3}{3\pi^2} \int_0^\infty \text{I2 }d\xi + \frac{2Tm_e^2}{3\pi^2} \int_0^{\infty} \text{dI2 } d\xi = \frac{8T^3}{3\pi^2} \int_0^\infty \frac{\xi^4}{\sqrt{\xi^2+x^2} \left(e^{\sqrt{\xi^2+x^2}}+1 \right)} d\xi + \frac{2Tm_e^2}{3\pi^2} \int_0^{\infty} \frac{3\xi^2}{\sqrt{\xi^2 + x^2} \left(e^{\sqrt{\xi^2 + x^2}}+1 \right)} d\xi . $$ 

Notice that we already have code for the integrand in the first term, ``I2``!! This function computes the *integrand* portion of the second term, ``dI2``. In addition, what is returned by this function has an additional $e^{\xi}$ as necessitated by the mathematical tool we use to evaluate this integral, Gauss-Laguerre quadrature (see above). The coefficients of both terms, $\left( 8T^3 \right)/\left(3\pi^2 \right)$ and $ \left(2Tm_e^2 \right)/\left(3\pi^2 \right)$ are both ultimately embedded into $dT/da$ below.
* **Inputs**: 
    * ``xi``: A dimensionless expression of $\xi = p/T$, and our dummy variable of integration. Notice that $dP_{e^{\pm}}$ is entirely a function of temperature (in thermal equilibrium) as momentum is integrated out.
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which $dP_{e^{\pm}}$ is being calculated.
* **Outputs**: 
    * ``dPressure``: The integrand at the given $\xi$ and $x$ value: $$ \frac{e^{\xi} 3\xi^2}{\sqrt{\xi^2 + x^2} \left(e^{\sqrt{\xi^2 + x^2}}+1 \right)} $$
    
### ``calculate_integral`` 
Uses a Gauss-Laguerre quadrature to integrate over one of the four integrands above, ``I1``, ``I2``, ``dI1``, or ``dI2``. Note that ``x_values``, as defined above, is the array of roots of the $n^{\rm th}$ Laguerre polynomial, while ``x`` is the adjusted temperature that is specific to each individual calculation. I couldn't really think of a more appropriate way to name either of them, but it is really only in this one particular instance that it's confusing.
* **Inputs**: 
    * ``I``: The function that returns the integrand over which we will be performing the integral. 
    * ``x``: The adjusted temperature, $x = m_e/T$, at the time step at which the integral, ``I``, is being calculated.
* **Outputs**: 
    * ``integral``: A summation over the integrand ``I`` using Gauss-Laguerre quadrature.
    
### ``trapezoid``
Performs a summation over a given array using trapezoid rule. Good for relatively simple integrals over smooth curves.
* **Inputs**: 
    * ``array``: Array of values that we will be integrating over, i.e., an array of some variable that depends on ``x``
    * ``dx``: Spaces between bins of a linearly spaced independent variable array
* **Outputs**: 
    * ``total``: The result of the summation: $$ \sum_{i=0}^{len(\text{array})-1} \frac{ dx * (\text{array}[i]+\text{array}[i+1])}{2} $$

## Decay rates

### ``rate1``
Calculates the decay rate of decay 1, $\nu_s \to \nu + \gamma$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 1, $$\Gamma_1 = \frac{9 G_F^2 \alpha m_s^5 \sin^2(\theta)}{512 \pi^4}$$

### ``rate2``
Calculates the decay rate of decay 2, $\nu_s \to \nu + \pi^0$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 2, $$\Gamma_2 = \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s (m_s^2-m_{\pi^0}^2) \sin^2(\theta)$$
    
### ``rate3``
Calculates the decay rate of decay 3, $\nu_s \to \pi^{\pm} + e^{\mp}$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 3, there is an extra 2 coefficient out front because $\nu_s$ can decay into either $\pi^+$ and $e^-$ OR $\pi^-$ and $e^+$, $$\Gamma_3 = 2 \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s \sqrt{(m_s^2 - (m_{\pi^0}+m_e)^2)*(m_s^2 - (m_{\pi^0}-m_e)^2)} \sin^2(\theta)$$ 

### ``rate4``
Calculates the decay rate of decay 4, $\nu_s \to \pi^{\pm} + \mu^{\mp}$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 4, there is an extra 2 coefficient out front because $\nu_s$ can decay into either $\pi^+$ and $\mu^-$ OR $\pi^-$ and $\mu^+$, $$\Gamma_4 = 2 \frac{G_F^2 f_{\pi}^2}{16 \pi} m_s \sqrt{(m_s^2 - (m_{\pi^{\pm}}+m_{\mu})^2)*(m_s^2 - (m_{\pi^{\pm}}-m_{\mu})^2)} \sin^2(\theta)$$ 
    
### ``rate5``
Calculates the decay rate of decay 5, $\nu_s \to 3\nu$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 5, $$\Gamma_5 = \frac{G_F^2 m_s^5 \sin^2(\theta)}{192 \pi^3}$$ 
    
### ``rate6``
Calculates the decay rate of decay 6, $\nu_s \to \nu + e^+ + e^-$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 6, $$\Gamma_6 = \frac{1}{3} \Gamma_5$$ 
    
### ``rate8``
Calculates the decay rate of decay 8, $\nu_s \to \nu + \mu^+ + \mu^-$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma``: The decay rate of decay 8, $$\Gamma_8 = \frac{1}{3} \Gamma_5$$ 
    
### ``ts``
Calculates the lifetime of the sterile neutrino (in units of MeV$^{-1}$) when only decay processes 1-4 are allowed. 
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``tau``: The overall lifetime of the sterile neutrino at the given mass and mixing angle in units of MeV$^{-1}$, $$\tau = \frac{1}{\Gamma_1 + \Gamma_2 + \Gamma_3 + \Gamma_4}$$
    
### ``ts_expanded``
Calculates the lifetime of the sterile neutrino (in units of seconds) when all decay processes (1-6 and 8) are allowed.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``tau``: The overall lifetime of the sterile neutrino at the given mass and mixing angle in units of seconds, $$\tau_s = \frac{6.582 \times 10^{22}}{\Gamma_1 + \Gamma_2 + \Gamma_3 + \Gamma_4 + \Gamma_5 + \Gamma_6 + \Gamma_8}$$
    
### ``decayrates``
Returns each of the decay rates as a given mass and mixing angle. The rate functions do not have a mechanism to prevent returning an imaginary number if a mass is passed to a decay rate that is not allowed (for example, a sterile neutrino with a mass less than 135 MeV can't decay via decay 2). This function prevents such an error by only calling decay rate functions if the decay is allowed by the mass of the sterile neutrino, and returns 0 for that rate otherwise.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``Gamma1``: $\Gamma_1$, allowed for all values of $m_s$
    * ``Gamma2``: $\Gamma_2$, allowed for $m_s>m_{\pi^0}$
    * ``Gamma3``: $\Gamma_3$, allowed for $m_s>m_{\pi^{\pm}}+m_e$
    * ``Gamma4``: $\Gamma_4$, allowed for $m_s>m_{\pi^{\pm}}+m_{\mu}$
    * ``Gamma5``: $\Gamma_5$, allowed for all values of $m_s$
    * ``Gamma6``: $\Gamma_6$, allowed for $m_s>2m_e$
    * ``Gamma8``: $\Gamma_8$, allowed for $m_s>2m_{\mu}$
    
### ``branching_ratios``
Calls ``decayrates`` and returns each divided by the total rate of decay.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``ratio1``: $\Gamma_1/\sum\Gamma$
    * ``ratio2``: $\Gamma_2/\sum\Gamma$
    * ``ratio3``: $\Gamma_3/\sum\Gamma$
    * ``ratio4``: $\Gamma_4/\sum\Gamma$
    * ``ratio5``: $\Gamma_5/\sum\Gamma$
    * ``ratio6``: $\Gamma_6/\sum\Gamma$
    * ``ratio8``: $\Gamma_8/\sum\Gamma$

## Relativity functions

### ``diracdelta`` 
In the decay of type $\nu_s \to B+C$, this function describes the monoenergtic probability distribution of the energy of particle $B$ in the rest frame of $\nu_s$
* **Inputs**: 
    * ``E``: Energy of current energy bin that we're looking at, always sends ``i*boxsize``
    * ``E_I``: Energy of particle $B$ in the rest frame of the parent particle
    * ``de``: Boxsize of current energy array
* **Outputs**: 
    * ``output``: If $E$ is within half a boxsize of $E_I$, then $E$ is in the energy bin that recieves all of the monoenergetic decay products of energy $E_I$, output = $\delta(E-E_{B})$
        
### ``diracdelta2``  
In the decay of type $\nu_s \to A \to B + C$, this function describes the energy distribution in the Home Frame ($\nu_s$ rest frame) of a particle that is monoenergetic in the Other Frame (particle $A$'s rest frame) (describes energy distribution of particles $B$ or $C$ in this scenario in the rest frame of $\nu_s$)
* **Inputs**: 
    * ``E``: Energy of current energy bin that we're looking at, always sends ``i*boxsize``
    * ``EBmin``: Maximum energy of particle $B$ in the Home Frame
    * ``EBmax``: Minimum energy of particle $B$ in the Home Frame
    * ``E_B``: Energy of particle $B$ in the Other Frame, i.e, energy of particle $B$ in the rest frame of particle $A$ from the monoenergetic decay of $A \to B+C$
    * ``v``: The speed of particle $A$'s rest frame relative to the sterile neutrino rest frame, $v = p_A/E_A$
    * ``gamma``: The Lorentz factor of particle $A$'s rest frame relative to the sterile neutrino rest frame, $\gamma = E_A/m_A $
* **Outputs**: 
    * ``tophat``: The probability of making a particle with energy $E$ in the rest frame of the sterile neutrino, $\dfrac{1}{2\gamma v E_B} \cases{1 \text{ if } E_{\rm min}<E<E_{\rm max} \\ 0 \text{ otherwise}}$
    
### ``EB`` 
In the decay of type $A \to B+C$ (where $B$ is monoenergetic), returns the energy of $B$ in the rest frame of $A$.
* **Inputs**: 
    * ``mA``: Mass of particle $A$ (decaying particle)
    * ``mB``: Mass of particle $B$ (decay product whose energy we're interested in)
    * ``mC``: Mass of particle $C$ (other decay product)
* **Outputs**: 
    * ``E_B``: Energy of particle $B$ in the rest frame of particle $A$, $$E_B = \frac{m_A^2 + m_B^2 - m_C^2}{2m_A}$$
    
### ``Gammamua``
Integral over differential decay rate of the muon in decay types III and IV in the rest frame of the muon. Lorentz transformed further in other parts of code to return to rest frame of the sterile neutrino, but this part of the code exclusively works with the three-product decay of the muon in the rest frame of the muon. The primes are there to remind us that this needs to be further transformed to be in the Home Frame.

So the purpose of this code is to take an neutrino energy bin in the rest frame of the mother particle of the muon, $E$, figure out the minimum and maximum neutrino energies ($a$ and $b$, respectively) that could Lorentz transform to $E$ from the rest frame of the muon. 
* **Inputs**: 
    * ``a``: Given that the current neutrino energy bin is $E$, the minimum possible neutrino energy in the rest frame of the muon that could Lorentz transform to $E$ in the rest frame of the mother particle of the muon is $a = \frac{E}{\gamma_{\mu}(1+v_{\mu})}$. 
    * ``b``: Given that the current neutrino energy bin is $E$, the maximum possible neutrino energy in the rest frame of the muon that could Lorentz transform to $E$ in the rest frame of the mother particle of the muon is $ \frac{E}{\gamma_{\mu}(1-v_{\mu})}$. However, the neutrinos from this decay process cannot have a energy greater than ``Enumax`` where $E_{\nu \rm max} = \frac{m_{\mu}}{2}(1-\epsilon_e^2)$, so we set $b = {\rm min}(E_{\rm max}', \frac{E}{\gamma_{\mu}(1-v_{\mu})})$.
* **Outputs**: 
    * ``Gam_mua``: The integral over the differential decay rate $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ in the rest frame of the muon, from the smallest possible energy of the neutrino that could Lorentz transform to $E$ in the mother particle rest frame, $a$, to the greatest possible energy of the neutrino that could Lorentz transform to $E$ in the mother particle rest frame, $b$, $$ \begin{aligned} \Gamma_{\mu_a}' &= \int^{b}_a \frac{d\Gamma_{\mu}'}{dE_{\nu}'} \frac{dE_{\nu}'}{E_{\nu}'} \\ &= \frac{8 G_F m_{\mu}^2}{16\pi^3} \int^b_a \frac{E_{\nu}'}{1-\frac{2E_{\nu}'}{m_{\mu}}} \left( 1 - \frac{2E_{\nu}'}{m_{\mu}} - \frac{m_e^2}{m_{\mu}^2} \right)^2 dE_{\nu}'  \\ &= \frac{-8 G_F m_{\mu}^2}{16\pi^3} \left. \left( \frac{\frac{1}{4} m_e^4 m_{\mu} \log{\left(|{2 E_{\nu}' - m_{\mu}}|\right)} + \frac{1}{6} E_{\nu}' \left(3 m_e^4 + 6E_{\nu}' m_e^2 m_{\mu} + m_{\mu}^2 E_{\nu}' (4E_{\nu}' - 3m_{\mu}) \right)}{m_{\mu}^3} \right) \right\vert_a^b \end{aligned} $$


### ``Gammamub``
Calculates integral over differential decay rate of the muon in decay types III and IV in the rest frame of the muon, but without the $E_{\nu}'$ in the denominator to isolate just the muon decay rate part of the integral. Lorentz transformed further in other parts of code to return to rest frame of the sterile neutrino, but this part of the code exclusively works with the three-product decay of the muon in the rest frame of the muon. The primes are there to remind us that this needs to be further transformed to be in the Home Frame.   
* **Inputs**: 
    * None
* **Outputs**: 
    * ``Gam_mub``: The integral over the differential decay rate $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}'$ from the smallest possible energy of the neutrino in the muon rest frame, 0, to the greatest possible energy of the neutrino in the muon rest frame, $E_{\rm max}'$, $$ \begin{aligned} \Gamma_{\mu_b}' &= \int^{E_{\rm max}'}_0 \frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}' \\ &= \frac{8 G_F m_{\mu}^2}{16\pi^3} \int^{E_{\rm max}'}_0 \frac{(E_{\nu}')^2}{1-\frac{2E_{\nu}'}{m_{\mu}}} \left( 1 - \frac{2E_{\nu}'}{m_{\mu}} - \frac{m_e^2}{m_{\mu}^2} \right)^2 dE_{\nu}'  \\ &= \frac{-8 G_F m_{\mu}^2}{16\pi^3} \left. \left(\frac{3 m_e^4 m_{\mu}^2 \log{\left(|{2 E_{\nu}' - m_{\mu}}|\right)} + 6 m_e^4 E_{\nu}' (m_{\mu}+E_{\nu}') + 16 m_e^2 m_{\mu} {E_{\nu}'}^3 + 4 m_{\mu}^2 {E_{\nu}'}^3 (3{E_{\nu}'} - 2m_{\mu})}{24 m_{\mu}^3} \right) \right\vert_0^{E_{\rm max}'} \end{aligned}$$
    
### ``Gam_mub``
Integral over $\frac{d\Gamma_{\mu}'}{dE_{\nu}'}dE_{\nu}'$, it's constant and only needs to be calculated once

### ``u_integral``
Using Gauss-Legendre quadrature, this function calculates the integral over all possible muon energies in the rest frame of the pion (for Decay Type IV) to Lorentz transform the active neutrino spectra into this rest frame.
* **Inputs**: 
    * ``E_mumin``: Minimum energy of the muon from the pion decay in the rest frame of the sterile neutrino
    * ``E_mumax``: Maximum energy of the muon from the pion decay in the rest frame of the sterile neutrino
    * ``Eactive``: Current neutrino energy bin in the Home Frame, ``i*boxsize``, $E$
* **Calculations** 
    * ``Eu_array``: Array of muon energies in the rest frame of the sterile neutrino that is created in accordance with Gauss-Legendre quadrature
    * ``gammau``: The Lorentz factor of the pion's rest frame to the rest frame of the sterile neutrino at the current energy bin
    * ``pu``: The momentum of the muon in the sterile neutrino rest frame at the current energy bin
    * ``vu``: The speed of the pion's rest frame relative to the sterile neutrino's rest frame at the current energy bin
    * ``Gam_mua``: The integral over the muon differential decay rate, $\Gamma_{\mu_a}' = \int_a^b \frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ where $a= E/(\gamma_{\mu}(1+v_{\mu}))$ and $b= {\rm min}(E_{\nu \rm max}, E/(\gamma_{\mu}(1-v_{\mu})))$.
* **Outputs**: 
    * ``integral``: Calculates the integral $$\int_{E_{\mu \rm min}}^{E_{\mu \rm max}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu}$$

# ODES 

The following functions are all seperate driver functions for decays 1, 2, 3, and 4. In other codes I think it will really be better to have them all smushed into one driver function, but this worked best here since the goal was to create a graph that dealt with each decay process individually. Here I will describe calculations for $dt/da$ and $dT/da$ since these occur in every driver function (which is obviously inefficient in the long run). Then, the description of each driver function will simply describe the parts of the driver unique to that decay process.

### ``dt/da``, changes in later code when $\epsilon$ is introduced

$$ \frac{dt}{da} = \frac{m_{\rm Pl}}{2 a} \frac{1}{\sqrt{\cfrac{2\pi}{3} \left( \cfrac{T^4 \pi^2}{15} + \cfrac{2T^4 \text{``I1``}}{\pi^2} + \cfrac{7 \pi^2 T_{\rm cm}^4}{40} + m_sn_s +  {\displaystyle \int } \cfrac{df}{da} \cfrac{1}{E^2 a^3} \right)}} $$


### ``dT/da``, first term in the numerator changes to $\frac{1}{T} \frac{dQ}{da}$ in later code

 $$\frac{dT}{da} = \cfrac{ \cfrac{f_T m_s a^3}{\tau_s T} n_s \cfrac{dt}{da} -3a^2T^3 \left[ \cfrac{4\pi^2}{45} + \cfrac{2}{\pi^2} \left( \text{``I1``} + \cfrac{1}{3} \text{``I2``} \right) \right]}{3T^2a^3 \left[ \cfrac{4\pi^2}{45} + \cfrac{2}{\pi^2} \left( \text{``I1``} + \cfrac{1}{3} \text{``I2``} \right) \right] - \cfrac{2 m_e^2 a^3}{\pi^2} \left( \text{``dI1``} + \cfrac{1}{3} \text{``dI2``} \right)} $$ 

## Decay 1

### ``driver1``
Decay 1: $$\nu_s \rightarrow \nu + \gamma$$
Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
    * ``ns``: Function that defines $n_s$, the number density of sterile neutrinos:
         * ### ``ns``
             * **Inputs**:
                 * ``Tcm``: Comoving temperature ($1/a$)
                 * ``t``: The time in units of MeV$^{-1}$
                 * ``ms``: Mass of the sterile neutrino 
                 * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
             * **Outputs**:
                 * ``n_s``: The number density of the sterile neutrino at the given time and comoving temperature (since $T_{\rm cm}$ describes the sterile neutrino population), $$ n_s = \frac{3D \zeta(3)}{2 \pi^2} T_{\rm cm}^3 e^{-t/\tau_s}$$
* **Calculations**:
    * ``E_B1``: The energy of the active neutrino decay product in the Home Frame, $E_{\nu1} = \frac{m_s}{2}$ 
    * ``dPdtdE1``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 1 in time interval $dt$, $$\left(\frac{dP}{dt \, dE}\right)_1 = \Gamma_1 \delta(E-E_{\nu1}) $$
* **Outputs**: 
    * ``a_array1``: An array of $a$ values (scale factor) as the code progressed. Steps between $a$ in the array are variable due to the variable step size algorithm in ``DES``, and the array stops either when ``a_final`` is reached or when ``len(a_array1)`` reaches ``steps``, whichever happens first. This should be identical to ``a_array2``, ``a_array3``, and ``a_array4``.
    * ``y_matrix1``: A matrix of the solutions to all the differential equations that correspond to the scale factor in ``a_array1``. That is, ``y_matrix1[i]`` is an array of the solutions to all the differential equations at ``a_array1[i]``. Specifically, ``y_matrix1[i][-2]`` is the temperature at ``a_array1[i]``, ``y_matrix1[i][-1]`` is the time at ``a_array1[i]``, and the rest of the values in ``y_matrix1[i]`` are the total number of neutrinos $N_E$ with energies between $E$ and $E+dE$ FROM DECAY 1 for all the energy bins at ``a_array1[i]`` (Later these dependent variables will be $f(\epsilon)$ instead of $N_E$). $$\frac{dN_E}{da} = \left(\frac{dP}{dt \, dE}\right)_1 n_s a^3 \frac{dt}{da}. $$ 

### ``derivatives1``
* **Inputs**: 
    * ``a``: Scale factor from previous time step 
    * ``y``: All dependent variables ($N_E$, temperature, and time) from previous time step
* **Outputs**: 
    * ``d_array1``: Derivatives at this previous time step that will be used to calculate ``a`` and ``y`` for the next time step

## Decay 2

### ``driver2``
Decay 2: $$\nu_s \rightarrow \nu + \pi^0$$ Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
    * ``ns``: Function that defines $n_s$, the number density of sterile neutrinos:
         * ### ``ns``
             * **Inputs**:
                 * ``Tcm``: Comoving temperature ($1/a$)
                 * ``t``: The time in units of MeV$^{-1}$
                 * ``ms``: Mass of the sterile neutrino 
                 * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
             * **Outputs**:
                 * ``n_s``: The number density of the sterile neutrino at the given time and comoving temperature (since $T_{\rm cm}$ describes the sterile neutrino population), $$ n_s = \frac{3D \zeta(3)}{2 \pi^2} T_{\rm cm}^3 e^{-t/\tau_s}$$
* **Calculations**:
    * ``E_B2``: The energy of the active neutrino decay product in the Home Frame, $E_{\nu2} = \frac{m_s^2 - m_{\pi^0}}{2m_s}$
    * ``dPdEdt2``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 2 in time interval $dt$, $$\left(\frac{dP}{dt \, dE}\right)_2 = \Gamma_2 \delta(E-E_{\nu2}) .$$ 
* **Outputs**: 
    * ``a_array2``: An array of $a$ values (scale factor) as the code progressed. Steps between $a$ in the array are variable due to the variable step size algorithm in ``DES``, and the array stops either when ``a_final`` is reached or when ``len(a_array2)`` reaches ``steps``, whichever happens first. This should be identical to ``a_array1``, ``a_array3``, and ``a_array4``.
    * ``y_matrix2``: A matrix of the solutions to all the differential equations that correspond to the scale factor in ``a_array2``. That is, ``y_matrix2[i]`` is an array of the solutions to all the differential equations at ``a_array2[i]``. Specifically, ``y_matrix2[i][-2]`` is the temperature at ``a_array2[i]``, ``y_matrix2[i][-1]`` is the time at ``a_array2[i]``, and the rest of the values in ``y_matrix2[i]`` are the total number of neutrinos $N_E$ with energies between $E$ and $E+dE$ FROM DECAY 2 for all the energy bins at ``a_array2[i]`` (Later these dependent variables will be $f(\epsilon)$ instead of $N_E$). $$\frac{dN_E}{da} = \left(\frac{dP}{dt \, dE}\right)_2 n_s a^3 \frac{dt}{da}. $$

### ``derivatives2``
* **Inputs**: 
    * ``a``: Scale factor from previous time step 
    * ``y``: All dependent variables ($N_E$, temperature, and time) from previous time step
* **Outputs**: 
    * ``d_array2``: Derivatives at this previous time step that will be used to calculate ``a`` and ``y`` for the next time step 

## Decay 3

### ``driver3``
Decay 3a: $$\nu_s \rightarrow \pi^{\pm} + e^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} $$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion. We're interested in the muon neutrino produced by the decay of the pion.

Decay 3b: $$\nu_s \rightarrow \pi^{\pm} + e^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion, Other Other Frame is rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
    * ``ns``: Function that defines $n_s$, the number density of sterile neutrinos:
         * ### ``ns``
             * **Inputs**:
                 * ``Tcm``: Comoving temperature ($1/a$)
                 * ``t``: The time in units of MeV$^{-1}$
                 * ``ms``: Mass of the sterile neutrino 
                 * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
             * **Outputs**:
                 * ``n_s``: The number density of the sterile neutrino at the given time and comoving temperature (since $T_{\rm cm}$ describes the sterile neutrino population), $$ n_s = \frac{3D \zeta(3)}{2 \pi^2} T_{\rm cm}^3 e^{-t/\tau_s}$$
* **Calculations referring to initial part of decay 3**:
    * ``E_pi3``: Energy of the charged pion from decay 3 in the Home Frame, $E_{\pi^{\pm}_3} = \frac{m_s^2 + m_{\pi^{\pm}}^2 - m_e^2}{2m_s}$
    * ``p_pi3``: Momentum of the charged pion from decay 3 in the Home Frame, $p_{\pi^{\pm}_3} = \sqrt{E_{\pi^{\pm}_3}^2 - m_{\pi^{\pm}}^2}$
    * ``v3``: The speed of the pion rest frame relative to the sterile neutrino rest frame, $v_3 = p_{\pi^{\pm}_3}/E_{\pi^{\pm}_3}$
    * ``gammapi3``: The Lorentz factor of the pion's rest frame relative to the sterile neutrino rest frame, $\gamma_{\pi3} = E_{\pi^{\pm}_3}/m_{\pi^{\pm}} $
* **Calculations referring to decay 3a**:
    * ``E_B3``: Energy of the muon neutrino from the decay of the pion in the Other Frame (rest frame of the pion), $E_{B3} = \frac{m_{\pi^{\pm}}^2-m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``E_B3max``: Maximum energy of the muon neutrino in the Home Frame, $E_{B3_{\rm max}} = \gamma_{\pi3} E_{B3}(1 + v_3)$
    * ``E_B3min``: Minimum energy of the muon neutrino in the Home Frame, $E_{B3_{\rm min}} = \gamma_{\pi3} E_{B3}(1 - v_3)$
    * ``dPdtdE3a``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 3a in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process the other half of the time, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{3a} = \frac{\Gamma_3}{2}  \dfrac{1}{2\gamma_{\pi3} v_3 E_{B3}} \cases{1 \text{ if } E_{B3_{\rm min}}<E<E_{B3_{\rm max}} \\ 0 \text{ otherwise}} $$
* **Calculations referring to decay 3b**:    
    * ``E_mu3``: The energy of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $E_{\mu_3} = m_{\pi^{\pm}} - E_{B3} = \frac{m_{\pi^{\pm}}^2+m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``p_mu3``: The momentum of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $p_{\mu_3} = \sqrt{E_{\mu_3}^2 - m_{\mu}^2}$
    * ``E_mumax3``: Maximum energy of the muon in the Home Frame, $E_{{\mu}_{3 \rm max}} = \gamma_{\pi3}(E_{\mu_3} + (v_3 p_{\mu_3}))$
    * ``E_mumin3``: Minimum energy of the muon in the Home Frame, $E_{{\mu}_{3 \rm min}} = \gamma_{\pi3}(E_{\mu_3} - (v_3 p_{\mu_3}))$
    * ``dPdtdE3b``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 3b in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{3b} = \frac{\Gamma_3}{2 \gamma_{\pi3} v_3 p_{\mu3} \Gamma_{\mu_b}'} \int_{E_{\mu \rm min}}^{E_{\mu \rm max}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu} $$
* **Outputs**: 
    * ``a_array3``: An array of $a$ values (scale factor) as the code progressed. Steps between $a$ in the array are variable due to the variable step size algorithm in ``DES``, and the array stops either when ``a_final`` is reached or when ``len(a_array3)`` reaches ``steps``, whichever happens first. This should be identical to ``a_array1``, ``a_array2``, and ``a_array4``.
    * ``y_matrix3``: A matrix of the solutions to all the differential equations that correspond to the scale factor in ``a_array3``. That is, ``y_matrix3[i]`` is an array of the solutions to all the differential equations at ``a_array3[i]``. Specifically, ``y_matrix3[i][-2]`` is the temperature at ``a_array3[i]``, ``y_matrix3[i][-1]`` is the time at ``a_array3[i]``, and the rest of the values in ``y_matrix3[i]`` are the total number of neutrinos $N_E$ with energies between $E$ and $E+dE$ FROM DECAY 3 for all the energy bins at ``a_array3[i]`` (Later these dependent variables will be $f(\epsilon)$ instead of $N_E$). $$\frac{dN_E}{da} = \left(\frac{dP}{dt \, dE}\right)_3 n_s a^3 \frac{dt}{da} $$ where $$\left(\frac{dP}{dt \, dE}\right)_3 = \left(\frac{dP}{dt \, dE}\right)_{3a} + \left(\frac{dP}{dt \, dE}\right)_{3b} .$$ 

### ``derivatives3``
* **Inputs**: 
    * ``a``: Scale factor from previous time step 
    * ``y``: All dependent variables ($N_E$, temperature, and time) from previous time step
* **Outputs**: 
    * ``d_array3``: Derivatives at this previous time step that will be used to calculate ``a`` and ``y`` for the next time step

## Decay 4

### ``driver4``
Initial part of Decay 4: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp}$$ Home Frame is simply the rest frame of the sterile neutrino, no Other Frame is considered.

Decay 4a: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.

Decay 4b: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} $$ Home Frame is the rest frame of the sterile neutrino, and the Other Frame is the rest frame of the pion. We're interested in the muon neutrino and produced by the decay of the pion.

Decay 4c: $$\nu_s \rightarrow \pi^{\pm} + \mu^{\mp} \rightarrow \nu_{\mu} + \mu^{\pm} \rightarrow e^{\pm} + \nu_{\mu} + \nu_{e}$$ Home Frame is the rest frame of the sterile neutrino, Other Frame is the rest frame of the pion, Other Other Frame is rest frame of the muon. We're interested in the muon neutrino and electron neutrino produced by the decay of the muon, which have the same spectra.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
    * ``ns``: Function that defines $n_s$, the number density of sterile neutrinos:
         * ### ``ns``
             * **Inputs**:
                 * ``Tcm``: Comoving temperature ($1/a$)
                 * ``t``: The time in units of MeV$^{-1}$
                 * ``ms``: Mass of the sterile neutrino 
                 * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
             * **Outputs**:
                 * ``n_s``: The number density of the sterile neutrino at the given time and comoving temperature (since $T_{\rm cm}$ describes the sterile neutrino population), $$ n_s = \frac{3D \zeta(3)}{2 \pi^2} T_{\rm cm}^3 e^{-t/\tau_s}$$
* **Calculations referring to initial part of decay 4**:
    * ``E_pi4``: Energy of the charged pion from decay 4 in the Home Frame, $E_{\pi^{\pm}_4} = \frac{m_s^2 + m_{\pi^{\pm}}^2 - m_{\mu}^2}{2m_s}$
    * ``p_pi4``: Momentum of the charged pion from decay 4 in the Home Frame, $p_{\pi^{\pm}_4} = \sqrt{E_{\pi^{\pm}_4}^2 - m_{\pi^{\pm}}^2}$
    * ``v4``: The speed of the pion rest frame relative to the sterile neutrino rest frame, $v_4 = p_{\pi^{\pm}_4}/E_{\pi^{\pm}_4}$
    * ``gammapi4``: The Lorentz factor of the pion's rest frame reltaive to the sterile neutrino rest frame, $\gamma_{\pi4} = E_{\pi^{\pm}_4}/m_{\pi^{\pm}} $
    * ``E_mu4i``: Energy of the muon produced directly by the decay of the sterile neutrino, $E_{\mu_{4i}} = m_s - E_{\pi^{\pm}_4}$
* **Calculations referring to decay 4a**:    
    * ``Gam_mua``: Decay rate of muon in the rest frame of the muon and the spectra of neutrinos created by this decay in the rest frame of the muon, $\Gamma_{\mu_a}' = \int_a^b \frac{d\Gamma_{\mu}'}{dE_{\nu}'}\frac{dE_{\nu}'}{E_{\nu}'}$ where $a = E/(\gamma_{\pi4}(1+v_4))$ and $b= {\rm min}(E_{\nu \rm max},E/(\gamma_{\pi4}(1-v_4)))$
    * ``dPdtdE4a``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4a in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4a} = \frac{\Gamma_4 \Gamma_{\mu_a}'}{2\gamma_{\pi4} v_4 \Gamma_{\mu_b}'}$$
* **Calculations referring to decay 4b**:    
    * ``E_B4``: Energy of the muon neutrino from the decay of the pion in the Other Frame (rest frame of the pion), $E_{B4} = \frac{m_{\pi^{\pm}}^2-m_{\mu}^2}{2m_{\pi^{\pm}}}$
    * ``E_B4max``: Maximum energy of the muon neutrino in the Home Frame, $E_{B4_{\rm max}} = \gamma_{\pi4} E_{B4}(1 + v_4)$
    * ``E_B4min``: Minimum energy of the muon neutrino in the Home Frame, $E_{B4_{\rm min}} = \gamma_{\pi4} E_{B4}(1 - v_4)$
    * ``dPdtdE4b``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4b in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process the other half of the time, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4b} = \frac{\Gamma_4}{2}\dfrac{1}{2\gamma_{\pi4} v_4 E_{B4}} \cases{1 \text{ if } E_{B4_{\rm min}}<E<E_{B4_{\rm max}} \\ 0 \text{ otherwise}} $$
* **Calculations referring to decay 4c**:    
    * ``E_mu4ii``: The energy of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $E_{\mu_{4ii}} = m_{\pi^{\pm}} - E_{B4} = \frac{m_{\pi^{\pm}}^2+m_{\mu}^2}{2m_{\pi^{\pm}}}$ 
    * ``p_mu4ii``: The momentum of the muon from the decay of the pion in the Other Frame (rest frame of the pion), $p_{\mu_{4ii}} = \sqrt{E_{\mu_{4ii}}^2 - m_{\mu}^2}$
    * ``E_mumax4``: Maximum energy of the muon in the Home Frame, $E_{{\mu}_{4 \rm max}} = \gamma_{\pi4}(E_{\mu_{4ii}} + (v_4 p_{\mu_{4ii}}))$
    * ``E_mumin4``: Minimum energy of the muon in the Home Frame, $E_{{\mu}_{4 \rm min}} = \gamma_{\pi4}(E_{\mu_{4ii}} - (v_4 p_{\mu_{4ii}}))$
    * ``dPdtdE4c``: The probability that a sterile neutrino produces an active neutrino with energy between $E$ and $E+dE$ via decay 4c in time interval $dt$. **Note!**: this does not include the antineutrino produced by this process, multiply by two to include the antineutrino: $$\left(\frac{dP}{dt \, dE}\right)_{4c} = \frac{\Gamma_4}{2 \gamma_{\pi4} v_4 p_{\mu4} \Gamma_{\mu_b}'} \int_{E_{{\mu}_{4 \rm min}}}^{E_{{\mu}_{4 \rm max}}} \frac{\Gamma_{\mu_a}'}{2\gamma_{\mu}v_{\mu}} dE_{\mu} $$
* **Outputs**: 
    * ``a_array4``: An array of $a$ values (scale factor) as the code progressed. Steps between $a$ in the array are variable due to the variable step size algorithm in ``DES``, and the array stops either when ``a_final`` is reached or when ``len(a_array4)`` reaches ``steps``, whichever happens first. This should be identical to ``a_array1``, ``a_array2``, and ``a_array3``.
    * ``y_matrix4``: A matrix of the solutions to all the differential equations that correspond to the scale factor in ``a_array4``. That is, ``y_matrix4[i]`` is an array of the solutions to all the differential equations at ``a_array4[i]``. Specifically, ``y_matrix4[i][-2]`` is the temperature at ``a_array4[i]``, ``y_matrix4[i][-1]`` is the time at ``a_array4[i]``, and the rest of the values in ``y_matrix4[i]`` are the total number of neutrinos $N_E$ with energies between $E$ and $E+dE$ FROM DECAY 4 for all the energy bins at ``a_array4[i]`` (Later these dependent variables will be $f(\epsilon)$ instead of $N_E$). $$ \frac{dN_E}{da} = \left(\frac{dP}{dt \, dE}\right)_4 n_s a^3 \frac{dt}{da} $$ where $$\left(\frac{dP}{dt \, dE}\right)_4 = \left(\frac{dP}{dt \, dE}\right)_{4a} + \left(\frac{dP}{dt \, dE}\right)_{4b} + \left(\frac{dP}{dt \, dE}\right)_{4c}.$$ 

### ``derivatives4``
* **Inputs**: 
    * ``a``: Scale factor from previous time step 
    * ``y``: All dependent variables ($N_E$, temperature, and time) from previous time step
* **Outputs**: 
    * ``d_array4``: Derivatives at this previous time step that will be used to calculate ``a`` and ``y`` for the next time step

## Calculating and Graphing Total Final Numbers of Neutrinos

### ``calc_final_num``
Returns $N_E$, the total number of neutrinos over energy, from each decay process as well as the total from all decay processes. Can easily be converted to number density, $F_E$, by dividing by $a^3$.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * ``num1``: Array of total number of neutrinos from decay 1 over energy that corresponds to ``e_array``
    * ``num2``: Array of total number of neutrinos from decay 2 over energy that corresponds to ``e_array``
    * ``num3``: Array of total number of neutrinos from decay 3 over energy that corresponds to ``e_array``
    * ``num4``: Array of total number of neutrinos from decay 4 over energy that corresponds to ``e_array``
    * ``numsum``: Array of total number of neutrinos from all decays over energy that corresponds to ``e_array``
    
### ``graph_final_num``
For a given mass and mixing angle of sterile neutrino, this function graphs $N_E$, the total number of neutrinos over energy, from each decay process as well as the total from all decay processes by calling ``calc_final_num``.
* **Inputs**: 
    * ``ms``: Mass of the sterile neutrino 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**:
    * None, simply creates the number density graph and saves it to the current directory as ``NumberDensity.pdf``.

## Graphing Branching Ratio and Lifetimes Contour Plot

### ``graph_BR``
Plots branching ratios of decay paths 1-4 and "others" (sum of decay paths 5, 6, and 8) over masses 50-500 MeV for a given mixing angle. The masses can be changes in ``DecayGraphs.py`` but we feel that our model really only is accurate for models in the mass range of 150-500 MeV or so, and the 50-150 MeV is included to show how it is inaccurate for those masses since decay processes 5, 6, and 8 take over in that mass regime.
* **Inputs**: 
    * ``mixangle``: Mixing angle of the sterile neutrino with the three active flavors
* **Outputs**: 
    * None, simply creates the branching ratio graph and saves it to the current directory as ``Branchingratio.pdf``.
    
### ``graph_mix_ms_t``
This function graphs a contour plot of lifetimes of sterile neutrino models over a range of masses and mixing angles as passed to the function by the user.
* **Inputs**: 
    * ``ms1``: Lower bound of sterile neutrino masses to be included in the contour plot
    * ``ms2``: Upper bound of sterile neutrino masses to be included in the contour plot
    * ``mix1``: Lower bound of mixing angles to be included in the contour plot
    * ``mix2``: Upper bound of mixing angles to be included in the contour plot
* **Outputs**:
    * None, simply creates the lifetime contour plot and saves it to the current directory as ``DecayTimes.pdf``.